In [1]:
import requests
from bs4 import BeautifulSoup

In [21]:
from collections import deque
import time

from bs4.element import Tag

t = Tag('a', name='start_link', attrs={'href': '/wiki/%D0%9A%D0%BE%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D1%8B', 
                                       'title': 'Корованы'})

links_dict = {}
q = deque([t])
print(q)

while(len(q) != 0):
    print(f'queue len: {len(q)}, parsed links: {len(links_dict)}')
    current_link = q.popleft()

    time.sleep(0.1)
    r = requests.get('https://neolurk.org' + current_link['href'])
    soup = BeautifulSoup(r.text, "html5lib").find('div', attrs={'class': 'mw-parser-output'})

    # Удаляем ссылки на категории
    for div in soup.find_all("div", {'class': 'catlinks'}): 
        div.decompose()
    
    # Ссылки на другие статьи лурка не имеют класса
    links = soup.findAll('a', attrs={'class': None})

    for link in links:
        if(link.has_attr('href') and link.has_attr('title')):
            if link['href'].startswith('/wiki/'):
                if(link['title'] not in links_dict):
                    links_dict[link['title']] = link['href']
                    if(link not in q):
                        q.append(link)
                    
            

deque([<start_link href="/wiki/%D0%9A%D0%BE%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D1%8B" title="Корованы"></start_link>])
queue len: 1, parsed links: 0
queue len: 676, parsed links: 676
queue len: 684, parsed links: 685
queue len: 1284, parsed links: 1286
queue len: 2342, parsed links: 2345
queue len: 3163, parsed links: 3167
queue len: 3480, parsed links: 3485
queue len: 3638, parsed links: 3644
queue len: 3666, parsed links: 3673
queue len: 4359, parsed links: 4367
queue len: 4644, parsed links: 4653
queue len: 4983, parsed links: 4993
queue len: 5295, parsed links: 5306
queue len: 5572, parsed links: 5584
queue len: 5599, parsed links: 5612
queue len: 5754, parsed links: 5768
queue len: 5756, parsed links: 5771
queue len: 5997, parsed links: 6013
queue len: 5997, parsed links: 6014
queue len: 6065, parsed links: 6083
queue len: 6201, parsed links: 6220
queue len: 6488, parsed links: 6508
queue len: 6531, parsed links: 6552
queue len: 6641, parsed links: 6663
queue len: 7159, parsed links: 7

KeyboardInterrupt: 

In [24]:
def clear(soup):
    # Удаляем картинки и видео справа
    for div in soup.find_all("div", {'class':['thumb tright', 'toc']}): 
        div.decompose()
    
    # Удаляем галерию внизу страницы
    for data in soup.find_all("li", {'class':'gallerybox'}): 
        data.decompose()
    
    # Удаляем таблицу с ссылками внизу страницы и заголовки
    for data in soup(['tbody', 'h1', 'h2', 'h3']): 
        data.decompose()

clear(soup)

In [34]:
import pandas as pd
from tqdm import tqdm

dataset = pd.DataFrame(columns=['title', 'link', 'text'])

for title in tqdm(links_dict):
    try:
        time.sleep(0.05)
        current_link = links_dict[title]
        r = requests.get('https://neolurk.org' + current_link)
        soup = BeautifulSoup(r.text, "html5lib").find('div', attrs={'class': 'mw-parser-output'})
        clear(soup)
        
        dataset.loc[len(dataset.index)] = [title, current_link, soup.get_text().replace('\n', ' ').replace('\xa0', ' ').strip()]
    except Exception as e:
        print(e)


  4%|██▋                                                                         | 733/20480 [07:07<2:53:36,  1.90it/s]

'NoneType' object has no attribute 'find_all'


  7%|████▉                                                                    | 1386/20480 [15:27<168:14:18, 31.72s/it]

'NoneType' object has no attribute 'find_all'


 12%|████████▊                                                                  | 2393/20480 [25:26<3:33:54,  1.41it/s]

...

'NoneType' object has no attribute 'find_all'


 23%|█████████████████▏                                                         | 4687/20480 [50:19<1:43:11,  2.55it/s]

'NoneType' object has no attribute 'find_all'


 51%|████████████████████████████████████▌                                   | 10392/20480 [1:47:23<1:44:15,  1.61it/s]


KeyboardInterrupt: 

In [35]:
dataset

,title,link,text
0,Старое обсуждение:Корованы,/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%BE%D0%B5_%D0...,"Внимание! Это старая дискуссия, которая некогд..."
1,Мунспик,/wiki/%D0%9C%D1%83%D0%BD%D1%81%D0%BF%D0%B8%D0%BA,"When European settlers first came upon Japan, ..."
2,Мем,/wiki/%D0%9C%D0%B5%D0%BC,"Раньше целый интернет говорил «превед, медвед»..."
3,Номад,/wiki/%D0%9D%D0%BE%D0%BC%D0%B0%D0%B4,Грабеж караванов — сущая ерунда— Сабж Номад...
4,Австралия,/wiki/%D0%90%D0%B2%D1%81%D1%82%D1%80%D0%B0%D0%...,"Австралия — единственное в мире государство, к..."
...,...,...,...
10380,КрАЗ,/wiki/%D0%9A%D1%80%D0%90%D0%97,"Прекрасная машина — это «КрАЗ», На два утюга п..."
10381,Мотороллер,/wiki/%D0%9C%D0%BE%D1%82%D0%BE%D1%80%D0%BE%D0%...,"Мотороллер — довольно известный мем Рунета, на..."
10382,Небесный пидарешт,/wiki/%D0%9D%D0%B5%D0%B1%D0%B5%D1%81%D0%BD%D1%...,Однажды Бандера ужинал с вояками УПА в карпатс...
10383,Флаг Украины,/wiki/%D0%A4%D0%BB%D0%B0%D0%B3_%D0%A3%D0%BA%D1...,"Флаг Украины — весьма специфический флаг, кото..."


In [36]:
dataset.to_csv('dataset.csv')